In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier

In [2]:
# Load data
df = pd.read_csv("C:/Users/cj.alonzo/OneDrive - Nice Systems Ltd/Documents/Personal/Data Science/personal projects/delays/merged_data_with_airlines.csv")

In [3]:
#made a new column, date
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20604118 entries, 0 to 20604117
Data columns (total 23 columns):
 #   Column               Dtype         
---  ------               -----         
 0   YEAR                 int64         
 1   MONTH                int64         
 2   DAY                  int64         
 3   DAY_OF_WEEK          int64         
 4   TAIL_NUM             object        
 5   OP_CARRIER_FL_NUM    int64         
 6   ORIGIN               object        
 7   ORIGIN_CITY_NAME     object        
 8   DEST                 object        
 9   DEST_CITY_NAME       object        
 10  DEP_TIME             object        
 11  DEP_DELAY            float64       
 12  DEP_DEL15            float64       
 13  ARR_TIME             object        
 14  ARR_DELAY            float64       
 15  ARR_DEL15            float64       
 16  CARRIER_DELAY        float64       
 17  WEATHER_DELAY        float64       
 18  NAS_DELAY            float64       
 19  SECURITY_DELAY     

In [4]:
#Convert to string and then to datetime
df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'].astype(str), format='%H:%M:%S', errors='coerce')

In [5]:
df['ARR_TIME'] = pd.to_datetime(df['ARR_TIME'].astype(str), format='%H:%M:%S', errors='coerce')

In [ ]:
df['DAY_OF_WEEK'] = df['Departure Date'].dt.dayofweek
df['MONTH'] = df['Departure Date'].dt.month

In [6]:
# Convert int64 to int32
int_columns = df.select_dtypes(include=['int64']).columns
df[int_columns] = df[int_columns].astype('int32')

# Convert float64 to float32
float_columns = df.select_dtypes(include=['float64']).columns
df[float_columns] = df[float_columns].astype('float32')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20604118 entries, 0 to 20604117
Data columns (total 23 columns):
 #   Column               Dtype         
---  ------               -----         
 0   YEAR                 int32         
 1   MONTH                int32         
 2   DAY                  int32         
 3   DAY_OF_WEEK          int32         
 4   TAIL_NUM             object        
 5   OP_CARRIER_FL_NUM    int32         
 6   ORIGIN               object        
 7   ORIGIN_CITY_NAME     object        
 8   DEST                 object        
 9   DEST_CITY_NAME       object        
 10  DEP_TIME             datetime64[ns]
 11  DEP_DELAY            float32       
 12  DEP_DEL15            float32       
 13  ARR_TIME             datetime64[ns]
 14  ARR_DELAY            float32       
 15  ARR_DEL15            float32       
 16  CARRIER_DELAY        float32       
 17  WEATHER_DELAY        float32       
 18  NAS_DELAY            float32       
 19  SECURITY_DELAY     

In [8]:
#80% missing bc they werent delayed
((df.isnull().sum() / len(df)) * 100).sort_values(ascending = False)

#dep del15 should have so drop
#we'll drop tail num

ARR_DEL15              2.156841
ARR_DELAY              2.156841
ARR_TIME               1.940379
DEP_DELAY              1.865768
DEP_DEL15              1.865768
DEP_TIME               1.864666
TAIL_NUM               0.413311
AIRLINE                0.000000
LATE_AIRCRAFT_DELAY    0.000000
SECURITY_DELAY         0.000000
NAS_DELAY              0.000000
WEATHER_DELAY          0.000000
CARRIER_DELAY          0.000000
YEAR                   0.000000
MONTH                  0.000000
DEST_CITY_NAME         0.000000
DEST                   0.000000
ORIGIN_CITY_NAME       0.000000
ORIGIN                 0.000000
OP_CARRIER_FL_NUM      0.000000
DAY_OF_WEEK            0.000000
DAY                    0.000000
DATE                   0.000000
dtype: float64

In [9]:
# Drop rows with missing values in the specified columns
columns_with_missing = ['ARR_DEL15', 'ARR_DELAY', 'ARR_TIME', 'DEP_DELAY', 'DEP_DEL15', 'DEP_TIME', 'TAIL_NUM']
df = df.dropna(subset=columns_with_missing)

# Verify the shape of the cleaned DataFrame
print("Original shape:", df.shape)
print("Shape after dropping rows with missing values:", df.shape)

Original shape: (20159720, 23)
Shape after dropping rows with missing values: (20159720, 23)


In [10]:
((df.isnull().sum() / len(df)) * 100).sort_values(ascending = False)

YEAR                   0.0
DEP_DEL15              0.0
AIRLINE                0.0
LATE_AIRCRAFT_DELAY    0.0
SECURITY_DELAY         0.0
NAS_DELAY              0.0
WEATHER_DELAY          0.0
CARRIER_DELAY          0.0
ARR_DEL15              0.0
ARR_DELAY              0.0
ARR_TIME               0.0
DEP_DELAY              0.0
MONTH                  0.0
DEP_TIME               0.0
DEST_CITY_NAME         0.0
DEST                   0.0
ORIGIN_CITY_NAME       0.0
ORIGIN                 0.0
OP_CARRIER_FL_NUM      0.0
TAIL_NUM               0.0
DAY_OF_WEEK            0.0
DAY                    0.0
DATE                   0.0
dtype: float64

In [11]:
# Define features and target variable
features = ['ORIGIN', 'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME', 'AIRLINE', 'YEAR', 'DAY_OF_WEEK', 'MONTH', 'TAIL_NUM']
X = df[features].copy()
y = df['DEP_DEL15']

In [12]:
# Perform label encoding for high-cardinality features
label_encoders = {}
for feature in ['ORIGIN', 'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME', 'AIRLINE', 'TAIL_NUM']:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature])
    label_encoders[feature] = le
    
# Perform one-hot encoding for low-cardinality features
low_cardinality_features = ['YEAR', 'DAY_OF_WEEK', 'MONTH']
X = pd.get_dummies(X, columns=low_cardinality_features)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [13]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [17]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [18]:
# Handle missing values in the target variable
imputer = SimpleImputer(strategy='constant', fill_value=0)  # Assuming 0 represents the majority class
y_train_imputed = imputer.fit_transform(y_train.values.reshape(-1, 1))

# Instantiate RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_imputed, y_train_imputed.ravel())

In [19]:
# Hyperparameter Tuning with RandomizedSearchCV
best_params = {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'subsample': 0.8}

param_distributions = {
    'learning_rate': [best_params['learning_rate']],
    'max_depth': [best_params['max_depth']],
    'subsample': [best_params['subsample']],
    'colsample_bytree': [best_params['colsample_bytree']],
}

In [20]:
xgb_model = XGBClassifier(eval_metric='logloss')
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_distributions,
                                   n_iter=5, cv=3, scoring='accuracy', random_state=42)
random_search.fit(X_train_resampled, y_train_resampled)

C:\Users\cj.alonzo\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=5,
                   param_distributions={'colsample_bytree': [0.8],
                                        'learning_rate': [0.2],
                                        'max_depth': [7], 'subsample': [0.8]},
                   random_state=42, scoring='accuracy')

In [21]:
# Evaluate the best model
best_xgb_model = random_search.best_estimator_
y_pred = best_xgb_model.predict(X_test_imputed)
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
print("Accuracy Score:", accuracy)
print("ROC-AUC Score:", roc_auc)

Accuracy Score: 0.6238427418634782
ROC-AUC Score: 0.6204257449361099


In [22]:
# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      0.63      0.73   3240999
         1.0       0.29      0.61      0.39    790945

    accuracy                           0.62   4031944
   macro avg       0.58      0.62      0.56   4031944
weighted avg       0.76      0.62      0.66   4031944

